In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import sklearn as sk
from sklearn import decomposition as dec
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.utils.prune as prune
import  sklearn.manifold as nonlin


In [47]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

mnist_train = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

trainloader_mnist =  torch.utils.data.DataLoader(mnist_train, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader_mnist =  torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                          shuffle=True, num_workers=2)



                                        


Files already downloaded and verified
Files already downloaded and verified


In [48]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=12, kernel_size=(5, 5))
        self.pool = nn.MaxPool2d(kernel_size=(2), stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=12, out_channels=16, kernel_size=(5, 5))
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [36]:
path = './cifar_net.pt'
model = nn.Module()
model = torch.load(path)
model.eval()

NN(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [88]:
path_mnist = './custom_nn_mnist.pt'
model_mnist = torch.load(path_mnist)
model_mnist.eval()


NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [89]:
random = torch.randn(256)

In [278]:
x_new_1 = model_mnist.fc1.weight.detach().numpy().T
iso = nonlin.Isomap(n_components=60)
# x_new_1 = x_new_1.reshape(60,2)
x_new_1 = iso.fit_transform(x_new_1)
# x_new_1 = x_new_1.flatten()
# random_matrix = random.reshape(256, 1)
# x_new_1_matrix = x_new_1.reshape(1,60)
print("x_new vector")
print(x_new_1.shape)

# new_weight_matrixIguess = np.matmul(random_matrix, x_new_1_matrix)



x_new vector
(256, 60)


In [322]:
uuh = torch.mm(model_mnist.fc1.weight, torch.tensor(x_new_1, dtype=torch.float))
print(uuh.shape)
wfc2 = torch.mm(model_mnist.fc2.weight, uuh)
print(wfc2.shape)
# wfc2 = model_mnist.fc2.weight.detach().numpy()
# wfc2 = wfc2.reshape(-1,60)
# isomap_ = nonlin.Isomap(n_components=30, n_neighbors=5)
# wfc2_reduced = isomap_.fit_transform(wfc2)
# wfc2_reduced = wfc2_reduced.reshape(-1, 60)

# print('second dimension reduced:')

# wfc2_reduced = torch.tensor(wfc2_reduced, dtype=torch.float32)
# print(wfc2_reduced.shape)

torch.Size([120, 60])
torch.Size([84, 60])


In [366]:
f = nn.Linear(256, 60, bias=False)
f2 = nn.Linear(60, 84, bias=False)
f3 = nn.Linear(84, 10, bias=False)
f.weight.data = torch.tensor(x_new_1.T, dtype=torch.float32)
f2.weight.data = wfc2
f3.weight = model_mnist.fc3.weight

g = nn.Linear(256, 120, bias=False)
g2 = nn.Linear(120, 84, bias=False)
g3 = nn.Linear(84,10,bias=False)
g.weight = model_mnist.fc1.weight
g2.weight = model_mnist.fc2.weight
g3.weight = model_mnist.fc3.weight


suka = torch.rand([60,84])
suka2 = torch.rand([120,84])
ff = f(random)
ff = f2(ff)
ff = f3(ff)
gg = g(random)
gg = g2(gg)
gg = g3(gg)

output1 = nn.functional.normalize(ff, dim=0)
output2 = nn.functional.normalize(gg, dim=0)

dist = torch.pairwise_distance(output1, output2)
print(dist)
ans = 1/(dist + 1) * 100
print(ans)

print("cosine similarity")
cos = nn.CosineSimilarity(dim=0)
print(cos(output1, output2) * 100)



tensor(0.3451, grad_fn=<NormBackward1>)
tensor(74.3421, grad_fn=<MulBackward0>)
cosine similarity
tensor(94.0442, grad_fn=<MulBackward0>)


In [51]:
x_new_1 = model_mnist.fc1(random)
x_new = model_mnist.fc2(x_new_1)
x_new = model_mnist.fc3(x_new)
print(x_new)


tensor([-0.5093, 10.7150, -0.0736, -8.4573,  7.4311, -0.9365, -1.6053,  2.1383,
        -8.8714, -2.8670], grad_fn=<AddBackward0>)


In [57]:
fc_1 = model_mnist.fc1.weight.detach().numpy().T
fa = nonlin.Isomap(n_components=60, n_neighbors=10)
fc_1_reduced = fa.fit_transform(fc_1)
fc_1_reduced = torch.tensor(fc_1_reduced.T,  dtype=torch.float32)
print(fc_1_reduced.shape)




torch.Size([60, 256])


In [54]:
# fc_2 = nn.functional.relu(model_mnist.fc2.weight)
fc_2 = model_mnist.fc2.weight.detach().numpy()
fc_2 = fc_2.reshape(-1,60)
isomap_ = nonlin.Isomap(n_components=30, n_neighbors=5)
fc_2_reduced = isomap_.fit_transform(fc_2)
fc_2_reduced = fc_2_reduced.reshape(-1, 120)
fa_ouput = nonlin.Isomap(n_components=21)
fc_2_reduced = fa_ouput.fit_transform(fc_2_reduced.T)
fc_2_reduced = fc_2_reduced.reshape(-1, 42)
fc_1_reduced = fc_2_reduced.T
print('second dimension reduced:')

fc_2_reduced = torch.tensor(fc_2_reduced, dtype=torch.float32)
print(fc_2_reduced.shape)



second dimension reduced:
torch.Size([60, 42])


## Testing just dimensional reduction 

In [55]:
fc_3 =  model_mnist.fc3.weight.detach().numpy().T
fc3_Isomap = nonlin.Isomap(n_components=5, n_neighbors=10)
fc_3_reduced = fc3_Isomap.fit_transform(fc_3)
fc_3_reduced = fc_3_reduced.reshape(-1, 10)
fc_3_reduced = torch.tensor(fc_3_reduced.T, dtype=torch.float32)
print(fc_3_reduced.shape)


torch.Size([10, 42])


In [59]:
model_mnist.fc1 = nn.Linear(256, 60,bias=True)
model_mnist.fc2 = nn.Linear(60, 42,bias=True)
model_mnist.fc3 = nn.Linear(42, 10,bias=True)

model_mnist.fc1.weight = nn.Parameter(fc_1_reduced)
model_mnist.fc2.weight = nn.Parameter(fc_2_reduced.T)
model_mnist.fc3.weight = nn.Parameter(fc_3_reduced)


In [117]:
print(x_new)

tensor([-1.6396,  1.5299,  3.4816, -1.8924,  0.8725, -1.7418, -1.1401,  3.4570,
         0.1371, -3.7845], grad_fn=<AddBackward0>)


In [27]:
x_new_2 = model_mnist.fc1(random)
x_new_2 = model_mnist.fc2(x_new_2)
x_new_2 = model_mnist.fc3(x_new_2)
print(x_new_2)


tensor([  8.2577, -18.1993,   9.7784,   1.0847, -15.9523,   6.3733,  26.0245,
        -10.8813,  -0.4002, -10.5237], grad_fn=<AddBackward0>)


In [67]:
total = 0
acc=0
correct = 0
for data in testloader_mnist:
    pics, lables = data
    output = model_mnist(pics)
    _, pred = torch.max(output.data, 1)
    correct += (pred == lables).sum().item()
    total += lables.size(0)

print('average acc:', 100 * correct/ total)



average acc: 97.19


In [190]:
model_mnist.eval()

NN(
  (conv1): Conv2d(1, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [62]:
from torch.profiler import profile, record_function, ProfilerActivity
import torch.optim as optim
running_loss = 0.0
total = 0.0
correct = 0.0
loss_nodr =[]
acc_nodr = []
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_mnist.parameters(), lr=0.001, momentum=0.9)

print(optimizer)




for epoch in range(2):
    
    print('epoch:', epoch)
    for i, data in enumerate(trainloader_mnist):
        
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad(True)
        with torch.set_grad_enabled(True):
            with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
             outputs = model_mnist(inputs).to(device)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        running_loss += loss.item()
        if i % 2000 == 1999:
            loss_temp = running_loss / 2000
            acc_temp = 100 * correct / total
            loss_nodr.append(loss_temp)
            acc_nodr.append(acc_temp)
            print(
                f'[{epoch + 1}, {i + 1:5d}] loss: {loss_temp:.3f} acc: {acc_temp:.3f}')
            running_loss = 0.0
            correct = 0.0
            total = 0.0
print('Finished Training')


SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
epoch: 0
[1,  2000] loss: 0.592 acc: 86.838
[1,  4000] loss: 0.199 acc: 94.162
[1,  6000] loss: 0.160 acc: 95.312
[1,  8000] loss: 0.130 acc: 96.325
[1, 10000] loss: 0.115 acc: 96.638
[1, 12000] loss: 0.113 acc: 96.625
[1, 14000] loss: 0.098 acc: 97.025
epoch: 1
[2,  2000] loss: 0.136 acc: 97.308
[2,  4000] loss: 0.090 acc: 97.425
[2,  6000] loss: 0.088 acc: 97.388
[2,  8000] loss: 0.082 acc: 97.525
[2, 10000] loss: 0.068 acc: 97.912
[2, 12000] loss: 0.072 acc: 98.025
[2, 14000] loss: 0.067 acc: 98.013
Finished Training


In [71]:
path = './custom_nn_mnist.pt'
torch.save(model_mnist, path)

In [30]:
path ='./custom_nn_mnist_isomap.pt'
torch.save(model_mnist, path)

In [79]:
def mfun(matrix, weight_matrix):
    for a in range(matrix.shape[0]):
        for b in range(matrix.shape[1]):
            matrix[a][b] = weight_matrix[a][b]
    return matrix


In [81]:
m1 = torch.zeros([60, 256])
m2 = torch.zeros([42, 60])
m3 = torch.zeros([10, 42])

m1 = mfun(m1, model_mnist.fc1.weight)
m2 = mfun(m2, model_mnist.fc2.weight)
m3 = mfun(m3, model_mnist.fc3.weight)

print(m1.shape)



torch.Size([60, 256])


In [87]:
lin1 = nn.Linear(in_features = 256, out_features=60,bias=True)
lin2 = nn.Linear(in_features = 60, out_features=42,bias=True)
lin3 = nn.Linear(in_features = 42, out_features=10,bias=True)

lin1.weight = model_mnist.fc1.weight
lin2.weight = model_mnist.fc2.weight
lin3.weight = model_mnist.fc3.weight

x_lin = lin1(random)
x_lin = lin2(x_lin)
x_lin = lin3(x_lin)
norm_xlin = nn.functional.normalize(x_lin, dim=0)
print(norm_xlin)
print(x_new)
ed = torch.pairwise_distance(norm_xlin, x_new)
print("distance")
print(ed)
result = 1/(1 + ed)
print(result * 100)
print("cosine similarity")
print(nn.CosineSimilarity(dim=0)(norm_xlin, x_new))



tensor([-0.1890, -0.5867,  0.5877, -0.1414, -0.0527, -0.2899,  0.3277,  0.0784,
         0.2266,  0.0545], grad_fn=<DivBackward0>)
tensor([-0.5093, 10.7150, -0.0736, -8.4573,  7.4311, -0.9365, -1.6053,  2.1383,
        -8.8714, -2.8670], grad_fn=<AddBackward0>)
distance
tensor(18.7919, grad_fn=<NormBackward1>)
tensor(5.0526, grad_fn=<MulBackward0>)
cosine similarity
tensor(-0.4186, grad_fn=<SumBackward1>)


In [81]:
lin1 = nn.Linear(in_features = 256, out_features=120,bias=False)
lin2 = nn.Linear(in_features = 120, out_features=84,bias=False)
lin3 = nn.Linear(in_features = 84, out_features=10,bias=False)

lin1.weight = model_mnist.fc1.weight
lin2.weight =  model_mnist.fc2.weight
lin3.weight =  model_mnist.fc3.weight

In [82]:
x_lin = lin1(random)
x_lin = lin2(x_lin)
x_lin = lin3(x_lin)
print(x_lin)


tensor([-0.9922,  1.0499,  1.0739, -2.9549,  1.8736,  0.2282,  0.8645,  0.9884,
         0.8530, -2.6438], grad_fn=<MvBackward0>)


# weight matrix multiplication testing

In [38]:
fc_1 = model_mnist.fc1.weight.detach().numpy().T
fa =nonlin.Isomap(n_components=60)
fc_1_m = fa.fit_transform(fc_1)
fc_1_m = torch.tensor(fc_1_m.T,  dtype=torch.float32)
print(fc_1_m.shape)



torch.Size([40, 256])


In [29]:
fc_1_m_new = torch.mm(model_mnist.fc1.weight.T, fc_1_m)
print(fc_1_m_new.shape)

torch.Size([256, 60])


In [39]:
# fc_2 = nn.functional.relu(model_mnist.fc2.weight)
# fc_2_x = torch.mm(new_matrix, model_mnist.fc2.weight.T)
# print(fc_2_x.shape)
# fc_2 = fc_2_x.detach().numpy()
# isomap_ = dec.PCA(n_components=42)
# fc_2_reduced_m = isomap_.fit_transform(fc_2)
# fc_2_reduced_m = fc_2_reduced_m.T
# print('second dimension reduced:')

# fc_2_reduced_m = torch.tensor(fc_2_reduced_m, dtype=torch.float32)
# print(fc_2_reduced_m.shape)
fc_2 = model_mnist.fc2.weight.detach().numpy()
fc_2 = fc_2.reshape(84 * 2, -1)
isomap_ = nonlin.Isomap(n_components=20)
reduced_fc_2 = isomap_.fit_transform(fc_2)
reduced_fc_2 = reduced_fc_2.reshape(-1, 20 * 2)
reduced_fc_2 = torch.tensor(reduced_fc_2,  dtype=torch.float32)
print(reduced_fc_2.shape)



torch.Size([84, 40])


In [45]:
lin_60 = nn.Linear(256, 40, bias=False)
lin_8460 = nn.Linear(40,84, bias=False)

lin_60.weight.data = fc_1_m
print(fc_1_m.shape)
lin_8460.weight.data = reduced_fc_2
output = lin_60(random)
output = lin_8460(output)
output = nn.functional.normalize(output, dim=0)
print(output)

lin_120 = nn.Linear(256, 120, bias=False)
lin_84 = nn.Linear(120,84, bias=False)
lin_120.weight.data = model_mnist.fc1.weight
lin_84.weight.data = model_mnist.fc2.weight

output2 = lin_120(random)
output2 = lin_84(output2)


print(output2)

print(nn.CosineSimilarity(dim=0)(output2, output) * 100)


dist = torch.pairwise_distance(output, output2)
print(dist)
result =1/(1 +dist)
print(result *100) 


torch.Size([40, 256])
tensor([ 0.0655, -0.2840,  0.0715, -0.1201,  0.0106, -0.1545, -0.0709, -0.1225,
        -0.2019,  0.0782,  0.1379,  0.0368,  0.1260, -0.1121, -0.0119,  0.0837,
        -0.0479, -0.1236,  0.1093, -0.0095, -0.1081,  0.1483, -0.1773, -0.0478,
         0.0687, -0.0505,  0.0256,  0.0311, -0.0546,  0.0593, -0.1113,  0.1878,
         0.0765, -0.2415, -0.1638, -0.0173,  0.1131,  0.1656,  0.1141, -0.1151,
        -0.0329,  0.0527,  0.0702, -0.1108,  0.1315,  0.0657, -0.1342, -0.0222,
         0.0230,  0.1309, -0.0738, -0.0666, -0.1203, -0.0908,  0.1100,  0.0595,
        -0.0408, -0.0239, -0.0839, -0.0709, -0.0875,  0.0174,  0.1358,  0.1235,
         0.0392, -0.1933,  0.0293, -0.1111,  0.0310, -0.0158,  0.0011,  0.1015,
         0.0247, -0.1355, -0.1582,  0.0930,  0.0432, -0.0255,  0.0324,  0.1620,
        -0.0675,  0.2043, -0.1831, -0.1480], grad_fn=<DivBackward0>)
tensor([ 1.5537,  0.9938, -0.3086, -1.5574,  0.6890, -1.6042, -0.0787, -0.3769,
         0.2473,  0.8930, -1.

In [11]:
new_martixfor_fc2 = torch.mm(fc_2_x.T, fc_2_reduced_m.T)
print(new_martixfor_fc2.shape)

torch.Size([84, 42])


In [12]:
fc_3_new = torch.mm(model_mnist.fc3.weight, new_martixfor_fc2)
print(fc_3_new.shape)
# fc_3_new = model_mnist.fc3.weight.detach().numpy().T
# fc3_Isomap = nonlin.Isomap(n_components=5, n_neighbors=10)
# fc_3_new_reduced = fc3_Isomap.fit_transform(fc_3_new)
# fc_3_new_reduced = fc_3_new_reduced.reshape(-1, 10)
# fc_3_new_reduced = torch.tensor(fc_3_new_reduced.T, dtype=torch.float32)
# print(fc_3_new_reduced.shape)


torch.Size([10, 42])


In [13]:
fc_3_new = torch.mm(fc_3_new, model_mnist.fc3.weight.T)
print(fc_3_new.shape)

torch.Size([42, 10])


In [229]:
print(fc_2_x.shape)

torch.Size([60, 84])


In [13]:
lin1_new = nn.Linear(in_features = 256, out_features=60,bias=True)
lin2_new= nn.Linear(in_features = 60, out_features=42,bias=True)
lin3_new = nn.Linear(in_features = 42, out_features=10,bias=True)

lin1_new.weight = nn.Parameter(fc_1_m)
lin2_new.weight = nn.Parameter(fc_2_reduced_m)
lin3_new.weight = nn.Parameter(fc_3_new)


In [19]:
mnist_lin1 = nn.Linear(in_features = 256, out_features=120,bias=True)
mnist_lin2 = nn.Linear(in_features = 120, out_features=84,bias=True)
mnist_lin3 = nn.Linear(in_features = 84, out_features=10,bias=True)

mnist_lin1.weight = model_mnist.fc1.weight
mnist_lin2.weight = model_mnist.fc2.weight
mnist_lin3.weight = model_mnist.fc3.weight

In [15]:
print(lin2_new.weight.shape)


torch.Size([42, 60])


In [21]:
a1 = lin1_new(random)
a1 = lin2_new(a1)
a1 = lin3_new(a1)
a1 = nn.functional.normalize(a1, dim=0)

a2 = mnist_lin1(random)
a2 = mnist_lin2(a2)
a2 = mnist_lin3(a2)
a2 = nn.functional.normalize(a2, dim=0)


print(a1)
print(a2)
ed = torch.pairwise_distance(a1, a2)
print(ed)
result =1/(1 +ed)
print(result *100) 

tensor([ 0.2364, -0.1304, -0.2433,  0.1608, -0.5722,  0.4288,  0.2356, -0.3944,
         0.3453, -0.0205], grad_fn=<DivBackward0>)
tensor([ 0.1291,  0.2988, -0.5994, -0.2388, -0.2318,  0.3710, -0.2537,  0.4554,
         0.0404, -0.1135], grad_fn=<DivBackward0>)
tensor(1.2903, grad_fn=<NormBackward1>)
tensor(43.6633, grad_fn=<MulBackward0>)
